In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd


In [2]:
df_SOI_village = gpd.read_file('/home/inductive_anks/Backup Files/CS and AI/Internship/Data/All-Inidia-SOI-Village-Shape-File/All-India-Villages-SOI.shp')

In [3]:
df_subdist_shape = gpd.read_file('/home/inductive_anks/Backup Files/CS and AI/Internship/Data/All-India-Subdistrict-Shape-File/SUBDISTRICT_BOUNDARY.shp')

In [4]:
df_SOI_village.tail()

,NAME,dtnmcoc,state,tehnmcoc,geometry
599737,thenthamaraikulam,kanniyakumari,tamil nadu,agastheeswaram,"POLYGON ((77.50307 8.11443, 77.50314 8.11415, ..."
599738,puthalam,kanniyakumari,tamil nadu,agastheeswaram,"POLYGON ((77.45023 8.11175, 77.45247 8.11075, ..."
599739,agasteeswaram,kanniyakumari,tamil nadu,agastheeswaram,"POLYGON ((77.51122 8.10797, 77.5154 8.10685, 7..."
599740,kanniyakumari,kanniyakumari,tamil nadu,agastheeswaram,"POLYGON ((77.54385 8.10316, 77.54398 8.10381, ..."
599741,thengamputhoor,kanniyakumari,tamil nadu,agastheeswaram,"POLYGON ((77.46212 8.08817, 77.46205 8.08842, ..."


In [5]:
df_SOI_village.rename(columns={'NAME':'VILLAGE'}, inplace=True)

In [6]:
df_SOI_village.head()

,VILLAGE,dtnmcoc,state,tehnmcoc,geometry
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101..."
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775..."
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418..."
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894..."
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541..."


In [7]:
df_subdist_shape.head()

,District,STATE,TEHSIL,Shape_Leng,Shape_Area,geometry
0,BIL>SPUR,HIMACHAL PRADESH,BHARARI,50253.228763,9.000306e+07,"POLYGON ((3690806.89 4831388.917, 3690871.421 ..."
1,BIL>SPUR,HIMACHAL PRADESH,BIL>SPUR,93446.685818,1.703525e+08,"POLYGON ((3699861.726 4812292.061, 3699965.144..."
2,BIL>SPUR,HIMACHAL PRADESH,GHURMW|N,105834.665205,1.906542e+08,"POLYGON ((3692609.535 4827764.245, 3692614.853..."
3,BIL>SPUR,HIMACHAL PRADESH,JHAND@TA,112831.475031,3.205376e+08,"POLYGON ((3667564.424 4819909.775, 3667603.174..."
4,BIL>SPUR,HIMACHAL PRADESH,NAMHOL,61752.616994,1.034979e+08,"POLYGON ((3701617.644 4803943.439, 3701666.832..."


## Filtering for Rajasthan

In [22]:
df_SOI_village = df_SOI_village.to_crs(epsg=4326)
df_subdist_shape = df_subdist_shape.to_crs(epsg=4326)


joined_df = gpd.sjoin(df_SOI_village, df_subdist_shape, how='left', predicate='within')

In [24]:
joined_df.head()

,VILLAGE,dtnmcoc,state,tehnmcoc,geometry,index_right,District,STATE,TEHSIL,Shape_Leng,Shape_Area
0,village point boundary,lahul & spiti,himachal pradesh,None,"POLYGON ((76.73598 33.18017, 76.73492 33.18101...",NaN,NaN,NaN,NaN,NaN,NaN
1,dhar chhoglu,chamba,himachal pradesh,pangi,"POLYGON ((76.55004 33.19772, 76.55082 33.19775...",NaN,NaN,NaN,NaN,NaN,NaN
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418...",11.0,CHAMBA,HIMACHAL PRADESH,P>NGI,188893.088214,1.490172e+09
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894...",11.0,CHAMBA,HIMACHAL PRADESH,P>NGI,188893.088214,1.490172e+09
4,rusmas,chamba,himachal pradesh,pangi,"POLYGON ((76.39866 33.17477, 76.39969 33.17541...",NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
villages_within_tehsil = joined_df.dropna(subset=['index_right'])

villages_not_within_tehsil = joined_df[joined_df['index_right'].isna()]

In [26]:
villages_within_tehsil.head()

,VILLAGE,dtnmcoc,state,tehnmcoc,geometry,index_right,District,STATE,TEHSIL,Shape_Leng,Shape_Area
2,dhar shinkal,chamba,himachal pradesh,pangi,"POLYGON ((76.52521 33.19414, 76.52596 33.19418...",11.0,CHAMBA,HIMACHAL PRADESH,P>NGI,188893.088214,1.490172e+09
3,sural bhatori,chamba,himachal pradesh,pangi,"POLYGON ((76.46151 33.17813, 76.46635 33.17894...",11.0,CHAMBA,HIMACHAL PRADESH,P>NGI,188893.088214,1.490172e+09
6,tai,chamba,himachal pradesh,pangi,"POLYGON ((76.43819 33.17915, 76.44485 33.17832...",11.0,CHAMBA,HIMACHAL PRADESH,P>NGI,188893.088214,1.490172e+09
7,chaloli,chamba,himachal pradesh,pangi,"POLYGON ((76.37752 33.16003, 76.38194 33.16366...",11.0,CHAMBA,HIMACHAL PRADESH,P>NGI,188893.088214,1.490172e+09
8,kuthah,chamba,himachal pradesh,pangi,"POLYGON ((76.37041 33.15255, 76.37295 33.15312...",11.0,CHAMBA,HIMACHAL PRADESH,P>NGI,188893.088214,1.490172e+09


In [27]:
villages_within_tehsil.shape

(441898, 11)

In [17]:
df_SOI_village.shape

(599742, 5)